In [ ]:
using DrWatson
@quickactivate

using IonChannelTools, Plots

include(srcdir("NaVB.jl"))

In [ ]:
dt = 1e-3 #time step in ms

In [ ]:
tL45 = IonChannelTools.pulse([0 20],dt,[0,0,-45,0])
d1 = IonChannelTools.evolvedist(Gmatrix,tL45[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"
ss = Array{Float64}(undef,0,9)
for i in 1:size(tL45,1)
    ss = [ss; IonChannelTools.steadystate(Gmatrix,tL45[i,2])]
end #all steady states
dss45 = IonChannelTools.fxnprotplot(d1-ss,tL45[:,2],tL45[:,1]) #difference of dist from steady state
dss45 = title!("distance from ss -45mV")

In [ ]:
Q45 = IonChannelTools.Qex(d1,Gmatrix,tL45[:,2],dt)
q45 = IonChannelTools.fxnprotplot(Q45,tL45[:,2],tL45[:,1])
q45 = Plots.title!("Qex for -45mV")

In [ ]:
S1 = []
for t in 1:length(tL45[:,1])
    append!(S1, IonChannelTools.S_tot(Gmatrix(-45),d1[t,:]))
end

In [ ]:
tL30 = IonChannelTools.pulse([0 20],dt,[0,0,-30,0])
d2 = IonChannelTools.evolvedist(Gmatrix,tL30[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"

tL15 = IonChannelTools.pulse([0 20],dt,[0,0,15,0])
d3 = IonChannelTools.evolvedist(Gmatrix,tL15[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"


S2 = []
for t in 1:length(tL30[:,1])
    append!(S2, IonChannelTools.S_tot(Gmatrix(-30),d2[t,:]))
end

S3 = []
for t in 1:length(tL15[:,1])
    append!(S3, IonChannelTools.S_tot(Gmatrix(15),d3[t,:]))
end

In [ ]:
S = plot(tL45[:,1],[S1 S2 S3],title="S_tot for -45mV, -30mV, 15mV",ylimits=[0,5])

In [ ]:
tL45l = IonChannelTools.pulse([0 200],dt,[0,0,-45,0])
d1l = IonChannelTools.evolvedist(Gmatrix,tL45l[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"
ssl = Array{Float64}(undef,0,9)
for i in 1:size(tL45l,1)
    ssl = [ssl; IonChannelTools.steadystate(Gmatrix,tL45l[i,2])]
end #all steady states

In [ ]:
dss45_long = IonChannelTools.fxnprotplot((d1l-ssl)[1:1000:end,:],tL45l[1:1000:end,2],tL45l[1:1000:end,1]) #difference of dist from steady state
dss45_long = Plots.title!("distance from steady state -45mV, 200ms long")

$$H[X] = -\sum_{x \in X} x\log{x}$$

In [ ]:
H_t = []
for i in 1:length(d1[:,1])
    append!(H_t, IonChannelTools.H(d1[i,:]))
end
H45 = plot(tL45[:,1],H_t)

ssH = IonChannelTools.H(ss[1,:])
maxH = IonChannelTools.H(fill(1/9,(1,9)))
H45 = hline!([ssH maxH])
H45 = title!("H[X], H[steady state], H[uniform dist]")

In [ ]:
H_tl = []
for i in 1:length(d1l[:,1])
    append!(H_tl, IonChannelTools.H(d1l[i,:]))
end
H45l = plot(tL45l[:,1],H_tl)

H45l = hline!([ssH maxH])
H45l = title!("longer H[X], H[steady state], H[uniform dist]")

In [ ]:
Q45 = IonChannelTools.Qex(d1,Gmatrix,tL45[:,2],dt)
dq45 = IonChannelTools.ddt(Q45,dt)
QH = dq45 + H_t

QH45 = plot(tL45[:,1],QH,title="dQex + H[X], and dS_tot",label="dQ+H",ylimits=[-1,5])
QH45 = plot!(tL45[:,1],dq45,label="dQ",linestyle=:dash)
QH45 = plot!(tL45[:,1],H_t,label="H",linestyle=:dash)
QH45 = plot!(tL45[:,1],S1,label="dS_tot")

makes sense, H[X] doesn't decay so the sum can't decay, wouldn't match S_tot behavior

In [ ]:
#trying with time derivative of H

QH2 = dq45 + IonChannelTools.ddt(H_t,dt)
QH452 = plot(tL45[:,1],QH2,title="dQex + dH[X], and dS_tot",label="dQ+H",ylimits=[-1,2])
QH452 = plot!(tL45[:,1],dq45,label="dQ",linestyle=:dash)
QH452 = plot!(tL45[:,1],IonChannelTools.ddt(H_t,dt),label="dH",linestyle=:dash)
QH452 = plot!(tL45[:,1],S1,label="dS_tot")

**trying a dynamic protocol again**

In [ ]:
dt = 1e-3
tLrs = IonChannelTools.spike(200,dt,"rs")

In [ ]:
dist_rs = IonChannelTools.evolvedist(Gmatrix,tLrs[:,2],dt)
Qrs = IonChannelTools.Qex(dist_rs,Gmatrix,tLrs[:,2],dt)

#steady states
ssrs = Array{Float64}(undef,0,9)
for i in 1:size(tLrs,1)
    ssrs = [ssrs; IonChannelTools.steadystate(Gmatrix,tLrs[i,2])]
end

#H for actual distribution
H_rs = []
for i in 1:length(dist_rs[:,1])
    append!(H_rs, IonChannelTools.H(dist_rs[i,:]))
end
#H for hypothetical steady states
H_rs_ss = []
for i in 1:length(dist_rs[:,1])
    append!(H_rs_ss, IonChannelTools.H(ssrs[i,:]))
end

In [ ]:
dSrs = []
for i in 1:length(tLrs[:,1])
    append!(dSrs, IonChannelTools.S_tot(Gmatrix(tLrs[i,2]),dist_rs[i,:]))
end

In [ ]:
#plot of H over protocol, with steady state H and max H
pHrs = IonChannelTools.fxnprotplot(H_rs,tLrs[:,2],tLrs[:,1])
pHrs = title!("H[X] with rs protocol")
pHrs = plot!(tLrs[:,1],H_rs_ss, label="H[ss]")
pHrs = hline!([maxH],label="max H")

In [ ]:
pQrs = IonChannelTools.fxnprotplot(Qrs,tLrs[:,2],tLrs[:,1])
pQrs = title!("Qex for rs")

In [ ]:
pdQrs = plot(tLrs[1:end-1,1],IonChannelTools.ddt(H_rs,dt)+IonChannelTools.ddt(Qrs,dt),label="dQ+dH",ylimits=[-.5,.5])
pdQrs = title!("dQex+dH, S_tot")
pdQrs = plot!(tLrs[1:end-1,1],IonChannelTools.ddt(H_rs,dt),label="dH",linestyle=:dash)
pdQrs = plot!(tLrs[1:end-1,1],IonChannelTools.ddt(Qrs,dt),label="dQ",linestyle=:dash)
pdQrs = plot!(tLrs[:,1],dSrs,label="dS_tot")

In [ ]:
S_s = IonChannelTools.S_array(IonChannelTools.S_sys,Gmatrix,tL45[:,2],d1) #system epr -45 mV
S_m = IonChannelTools.S_array(IonChannelTools.S_med,Gmatrix,tL45[:,2],d1) #med epr -45 mV

In [ ]:
S_check = plot(tL45[:,1],(S_s.+S_m), label="S_tot (sum)",title="S compenents -45mV")
S_check = plot!(tL45[:,1],[S_s S_m],linestyle=:dash,label=["S_sys" "S_med"])
#S_check = plot!(tL45[:,1],S1,linestyle=:dash,label="S_tot (direct)")
S_check = plot!(tL45[1:end-1,1],IonChannelTools.ddt(H_t,dt),label="dH")

In [ ]:
#critical points match between dH and S_sys
print("minimums for S_sys, dH \n")
print(findmin(S_s))
print("\n")
print(findmin(IonChannelTools.ddt(H_t,dt)))

print("\n\n left zeros for S_sys,dH \n")
print(findmin(abs.(S_s[1:findmin(S_s)[2]])))
print("\n")
print(findmin(abs.(IonChannelTools.ddt(H_t,dt)[1:findmin(S_s)[2]])))

#print("\n\n right zeros for S_sys,dH (index value is as shown +708 )\n")
#print(findmin(abs.(S_s[708:5000])))
#print("\n")
#print(findmin(abs.(ddt(H_t,dt)[708:5000])))